## Module 8: Confidence Intervals
### 08-01: T confidence Intervals

- In the previous we discussed creating a confidence interval using the CLT
    - They took the form Est $\pm$ ZQ $\times$ $\text{SE}_{\text{Est}}$
- Now we discuss T confidence interval, which repaces Z quantiles $ZQ$ by Student T quantiles $TQ$, which is useful for small sized samples
    - Est $\pm$ TQ $\times$ $\text{SE}_{\text{Est}}$
    - T statistic has thicker tails than the normal, therefore wider confidence intervals for small samples. For larger sized samples, it approaches the normal Z statistic

- Is indexed by a degrees of freedem; gets more like a standard normal as df gets larger
- It assumes that the underlying data are iid Gaussian with the result that
$\frac{\bar{X} - \mu}{S/\sqrt{n}}$ follows t distribution with $n-1$ dof.
- **Interval** is $\bar{X} \pm t_{n-1} S / \sqrt{n}$, where $t_{n-1}$ is the relevant quantile

**Notes about the t interval**
- The t interval technicall assumes that the data are iid normal, tough it is robust to this assumption
- It works well whenever the distribution of the data is roughly symmetric and mound shaped
- Paired observations are often analyzed using the t interval by taking differences
- For large degrees of freedom, t quantiles become the same as standard normal quantiles; therefore this interval converges to the same interval as the CLT yielded
- For skewed distributions, the spirit of the t interval assumptions are violated
    - Also, for skewed distributions, it doesnt make a lot of sense to center the interval at the mean
    - In this case, consider taking logs or using a different summary like the median
- For highly discrete data, like binary or Poisson, other intervals are available

### 08-02: T confidence intervals example

In [9]:
library(stats)
data(sleep)
head(sleep)

,extra,group,ID
,<dbl>,<fct>,<fct>
1,0.7,1,1
2,-1.6,1,2
3,-0.2,1,3
4,-1.2,1,4
5,-0.1,1,5
6,3.4,1,6


In [29]:
g1 <- sleep$extra[1:10]
g2 <- sleep$extra[11:20]
difference <- g2 - g1
mn <- mean(difference)
s <- sd(difference)
n <- 10

In [14]:
mn + c(-1,1) * qt(.975, n-1) * s /sqrt(n)

[1] 0.7001142 2.4598858

In [15]:
t.test(difference)


	One Sample t-test

data:  difference
t = 4.0621, df = 9, p-value = 0.002833
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 0.7001142 2.4598858
sample estimates:
mean of x 
     1.58 


In [16]:
t.test(g2, g1, paired = TRUE)


	Paired t-test

data:  g2 and g1
t = 4.0621, df = 9, p-value = 0.002833
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.7001142 2.4598858
sample estimates:
mean of the differences 
                   1.58 


In [17]:
t.test(extra ~ I(relevel(group, 2)), paired = TRUE, data = sleep)


	Paired t-test

data:  extra by I(relevel(group, 2))
t = 4.0621, df = 9, p-value = 0.002833
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.7001142 2.4598858
sample estimates:
mean of the differences 
                   1.58 


### 08-03: Independent group T intervals

- Supose that we want to compare the mean blood pressure between 2 groups in a randomized trial; those who received the treatment to those who received a placebo
- We cannot use the paired t test because the groups are independent and may have different sample sizes
- A $\left( 1-\alpha \right) \times 100\%$ confidence interval for $\mu_y - \mu_x$ is  $\bar{Y} - \bar{X} \pm t_{\left( n_{x} + n_{y}-2, \, (1-\alpha/2) \right)} S_{p} \left(\frac{1}{n_{x}} +  \frac{1}{n_{y}}\right)^{1/2}$
- **Variance estimator (pooled variance)**: $S_{p}^{2} = \left\{ (n_{x} - 1 ) S_{x}^{2} + (n_{y}-1 ) S_{y}^{2} \right\} / (n_{x} + n_{y} - 2)$
- Remember the interval is assuming a constant variance across the two groups
- If there is some doubt, assume a different variance per group, to be discussed later

*Example*
- Comparing SBP for 8 oral contraceptive users versus 21 controls
- $\bar{X}_{OC}$ = 132.86 mmHg with $s_{OC}$ = 15.34 mmHg
- $\bar{X}_{C}$ = 127.44 mmHg with $s_{C}$ = 18.23 mmHg
- Pooled variance estimate

In [20]:
sp <- sqrt(((8-1) * 15.34^2 + (21-1) * 18.23^2)/(8 + 21 - 2))
(132.86-127.44) + c(-1,1) * qt(.975, (8+21-2)) * sp * sqrt(1/8 + 1/21)

[1] -9.521097 20.361097

*Example*: Mistakenly treating the sleep data as grouped

In [27]:
sd(g1)

[1] 1.78901

In [30]:
n1 <- length(g1)
n2 <- length(g2)
sp <- ((n1-1) * sd(x1)^2 + (n2-1) * sd(x2)^2) / (n1 + n2 - 2)
md <- mean(g2) - mean(g1)
semd <- sp * sqrt(1 / n1 + 1 / n2)
rbind(
    md + c(-1,1) * qt(0.975, n1+n2-2) * semd,
    t.test(g2, g1, paired = FALSE, var.equal = TRUE)$conf,
    t.test(g2, g1, paired = TRUE)$conf
)

ERROR: Error in is.data.frame(x): object 'x1' not found


*Example*:ChickWeight data in R

In [54]:
#?dcast

In [44]:
library(datasets)
library(reshape2)
library(dplyr)
data(ChickWeight)
# define weight gain or loss
wideCW <- dcast(ChickWeight, Diet + Chick ~ Time, value.var='weight')
names(wideCW)[-(1:2)] <- paste("time", names(wideCW)[-(1:2)], sep = "")
wideCW <- mutate(wideCW
                , gain = time21- time0
                )

In [45]:
head(wideCW)

,Diet,Chick,time0,time2,time4,time6,time8,time10,time12,time14,time16,time18,time20,time21,gain
,<fct>,<ord>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,18,39,35,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,1,16,41,45,49,51,57,51,54,NA,NA,NA,NA,NA,NA
3,1,15,41,49,56,64,68,68,67,68,NA,NA,NA,NA,NA
4,1,13,41,48,53,60,65,67,71,70,71,81,91,96,55
5,1,9,42,51,59,68,85,96,90,92,93,100,100,98,56
6,1,20,41,47,54,58,65,73,77,89,98,107,115,117,76


In [46]:
head(ChickWeight)

,weight,Time,Chick,Diet
,<dbl>,<dbl>,<ord>,<fct>
1,42,0,1,1
2,51,2,1,1
3,59,4,1,1
4,64,6,1,1
5,76,8,1,1
6,93,10,1,1


In [53]:
wideCW14 <- subset(wideCW, Diet %in% c(3,4))
rbind(
    t.test(gain ~ Diet, paired = FALSE, var.equal = TRUE, data = wideCW14)$conf
    , t.test(gain ~ Diet, paired = FALSE, var.equal = FALSE, data = wideCW14)$conf
)

-26.20682,89.87349
-25.30020,88.96686


### 08-04: A note on uneqal variances

Use argument `var.equal = FALSE` in `t.test()` function

## Hypothesis testing
### 09-01: Hypothesis testing

- Hypothesis testing is concerned with making decisions using data
- A **null hypothesis** is specified that represents the status quo, usually habeled $H_0$
- The **null hypothesis** is assumed true and statistical evidence is requied to reject it in favor of a research or alternative hypothesis

*Example*
- A respiratory (呼吸的) disturbance index (RDI) of more than 30 events/hour, say, is considered evidence of severe sleep disordered brathing (SDB)
- Suppose that in a sample of 100 overweight subjects with other risk factors for SDB at a sleep clinic, the mean RDI was 32 events / hour with a standard deviation of 10 events / hour
- We might want to test the hypothesis that
    - $H_0$: $\mu$ = 30
    - $H_{a}$: $\mu > 30$
    - where $\mu$ is the population mean RDI

**Outcomes**
- The alternative hypotheses are typically of the form <, >, or $\neq$
- Note that there are 4 possible outcomes of our statistical decision process

|  TRUTH   | DECIDE  |  RESULT  |
|  ----  | ----  | ---- |
| $H_{0}$  | $H_{0}$ | Correctly accpet null |
| $H_{0}$  | $H_{a}$ | Type I error: **REJECT** a **TRUE** null hypothesis -- We fail to reject $H_0$:  $H_0$ could be true or we just dont have enough data to reject it |
| $H_{a}$  | $H_{a}$ | Correctly reject null |
| $H_{a}$  | $H_{0}$ | Type II error: **ACCEPT** a **FALSE** null hypothesis $H_0$ |

- Consider a court of law: the null hypothesis is that the defendant is innocent
- We require a standard on the available evidence to reject the null hypothesis (convict)
- If we set a low standard, then we would increase the percentage of innocent people convicted (**type I errors**); however we would also increase the percentage of guilty people convicted (**correctly rejecting the null**)
- If we set a high standard, then we increase the percentage of innocent people let free (**correctly accepting the null**) while we would also increase the percentage of guilty people let free (**type II errors**)

### 09-02: Example of choosing a rejection region
*Example*
- Sleep data again
- A reasonable strategy would reject the null hypothesis if $\bar{X}$ was larger than some constant, C
- Typically, C is chosen so that the probability of a Type I error, $\alpha$ is 0.05 (or some other relevant constant)
- $\alpha$ = Type I error rate = Probability of rejecting the null hypothesis when, in fact, the hypothesis is correct

*Example*
- standard error of the mean $10/\sqrt{100} = 1$
- Under $H_0: \, \bar{X} \sim \mathcal{N} \left( 30, 1 \right)$
- We want to choose $C$ so that the $P(\bar{X} > C; H_0)$ is 5%
- The 95th percentile of a normal distribution is 1.645 standard deviations from the mean
- If $C = 30 + 1 \times 1.645 = 31.645$, we will achive a cut point, so that the probability that a randomly drawn mean from this population is larger than this, is 5%
    - Then the probability that a $\mathcal{N}(30, 1)$ is larger than it, is $5\%$
    - So the rule "Reject $H_{0}$ when $\bar{X} \ge 31.645$" has the property that the probability of rejection is 5% when $H_{0}$ is true (for the $\mu_{0}, \, \sigma$ and $n$ given)
    
**Discussion**
- In general we dont convert C back to the original scale
- We would just reject because the $Z-score$, which is how many standard errors the sample mean is above the hypothesized mean   
$(32-30)/(10/\sqrt{100}) = 2$  
is greater than 1.645
- Or, whenever $\sqrt{n} \left( \bar{X} - \mu_{0} \right) / s > Z_{1-\alpha}$

In [1]:
#?qt

### 09-02: T tests
- Consider our example again. Suppose $n=16$ (rather than 100)
- The statistic  
$\frac{\bar{X} - 30}{s / \sqrt{16}}$  
follows a T distribution with 15 df under $H_0$
- $H_0: \mu=30$, $H_a: \mu > 30$
- Under $H_0$ the probability that it is larger than the 95th percentile of the T distribution is 5%
- The 95th percentile of the T distribution with 15 df is 1.7531 `qt(.95, 15)`
- So that our test statistic is now $\sqrt{16} (32-30) / 10 = 0.8$
- We now fail to reject, since 0.8 < 1.75

**Two sided tests**
- Suppose that we would reject the null hypothesis if in fact the mean was too large or too small
- i.e. we want to test the alternative $H_{a}: \mu \neq 30$
- We will reject if the test statistic 0.8, is either too large or too small
- Then we want the probability of rejecting under the null to be 5%, split equally as 2.5% in the upper tail and 2.5 in the lower tail
- Thus we reject if our test statistic is larger than `qt(0.975, 15)` or smaller than `qt(0.025, 15)`
    - This is the same as saying: reject if the absolute value of our statistic is larger than `qt(0.975, 15) = 2.1314
    - So we fail to reject the two sided test as well
    - (if you fail to reject the one sided test, you know that you will fail to reject the two sided)

#### T test in R

In [2]:
library(UsingR)
data(father.son)
# compare father's height, paired with son's height
t.test(father.son$sheight - father.son$fheight)
# equivalent to 
t.test(father.son$sheight, father.son$fheight, paired = TRUE)

Loading required package: MASS

Loading required package: HistData

Loading required package: Hmisc

Loading required package: lattice

Loading required package: survival

Loading required package: Formula

Loading required package: ggplot2


Attaching package: ‘Hmisc’


The following objects are masked from ‘package:base’:

    format.pval, units



Attaching package: ‘UsingR’


The following object is masked from ‘package:survival’:

    cancer





	One Sample t-test

data:  father.son$sheight - father.son$fheight
t = 11.789, df = 1077, p-value < 2.2e-16
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 0.8310296 1.1629160
sample estimates:
mean of x 
0.9969728 



	Paired t-test

data:  father.son$sheight and father.son$fheight
t = 11.789, df = 1077, p-value < 2.2e-16
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.8310296 1.1629160
sample estimates:
mean of the differences 
              0.9969728 


### 09-04: Two group testing
**Connections with confidence intervals**
- Consider testing $H_0: \mu = \mu_{0}$ versus $H_a: \mu \neq \mu_{0}$
- Take the set of all possible values for which you fail to reject $H_{0}$, this set is a $(1 - \alpha) 100\%$ confidence interval for $\mu$
- The same works in reverse; if a $(1-\alpha) 100\%$ interval contains $\mu_{0}$, then we *fail* to reject $H_{0}$  

**Two group intervals**  
- First, now you know how to do 2 group T tests since we already convered independent group T intervals
- Rejection rules are the same
- Test $H_{0}: \mu_{1} = \mu_{2}$
- Let's just go through an example


*Exact binomial test*
- Recall this problem, suppose a friend has 8 children, 7 of which are girls and none are twins
- Perform the relevant hypothesis test. $H_0 : p = 0.5$, $H_{a}:p > 0.5$
    - What is the relevant rejection region so that the probability of rejecting is (less than) 5%?

## Module 10: P values

P-values are a convenient way to communicate the results of a hypothesis test. When communicating a P-value, the reader can perform the test at whatever Type I error rate that they would like. Just compare the P-value to the desired Type I error rate and if the P-value is smaller, reject the null hypothesis.

Formally, the P-value is the probability of getting data as or more extreme than the observed data in favor of the alternative. The probability calculation is done assuming that the null is true. In other words if we get a very large T statistic the P-value answers the question "How likely would it be to get a statistic this large or larger if the null was actually true?". If the answer to that question is "very unlikely", in other words the P-value is very small, then it sheds doubt on the null being true, since you actually observed a statistic that extreme.

**P-values**
- Most common measure of statistical significance
- Their ubiquity, along with concern over their interpretation and use makes them controversial among statisticians
- **Idea**: Suppose nothing is going on - how unusual is it to see the estimate we got?
- **Approach**: 3 steps  
    - Define the hypothetical distribution of a data summary(statistic) when "nothing is going on" (null hypothesis)
    - Calculate the summary/statistic with the data we have (test statistic)
    - Compare what we calculated to our hypothetical distribution and see if the value is "extreme" (p-value)
    - If the p value is small, then the probability of observing a test statistic as extreme as we saw, is low if the null hypothesis were true

**P-values**
- Definition: the P-value is the probability under the null hypothesis of obtaining evidence as extreme or more extreme than that obtained
- If the p value is small, then either $H_{0}$ is true, and we have observed a rare event, or $H_{0}$ is false
- *Example*: Suppose that you get a T statistic of 2.5 for 15 df testing: $H_{0}: \mu = mu_{0}$ versus $H_{a}: \mu > \mu_{0}$
    - What is the probability of getting a T statistic as large as 2.5? --> 1.2%

In [62]:
pt(2.5, 15, lower.tail = FALSE)

[1] 0.0122529

**The Attained significance level**
- Our test statistic was 2 for $H_{0}: \mu = 30$ versus $H_{a}: \mu > 30$
- Notice that we rejected the one sided test when $\alpha = 0.05$, would we reject if $\alpha = 0.01$, how about $0.001$?

**Notes**
- By reporting a P-value the reader can perform the hypothesis test at whatever $\alpha$ level he or she chooses
- If the P-value is less than $\alpha$ you reject the null hypothesis
- If the P-value is greater than $\alpha$ you fail to reject the null hypothesis
- For two-sided hypothesis test, double the smaller of the two one-sided hypothesis test P-values

### 10-02: P-value further examples
*Example*: suppose a friend has 8 children, 7 of which are girls and none are twiens. If each gender has an independent 50% probability for each birth, what's the probability of getting 7 or more girls out of 8 births?
- Null hypothesis & alternative: $H_{0}: p = 0.5$ versus $H_{a}: p > 0.5$

In [63]:
choose(8,7) * 0.5^8 + choose(8,8) * 0.5^8

[1] 0.03515625

In [66]:
pbinom(6, size = 8, prob = 0.5, lower.tail = FALSE)

[1] 0.03515625

we would reject at 5% level  
we would reject at 4% level  
we would not reject at an type 1 error rate of 3%  

2-sided problem: $H_{0}: p = 0.5$ versus $H_{a}: p \neq 0.5$
- calculate 2 one-sided p-values 
    - the probability of being 7 or larger
    - the probability of being 7 or smaller
- take those 2 one-sided p-values, choose the smaller one, and double it

*Poisson example*
- Suppose that a hospital has an infection rate of 10 infections per 100 person/days at risk (rate of 0.1) during the last monitoring period
- Assume that an infection rate of 0.05 is an important benchmark
- Given the model, could the observed rate being larger than 0.05 be attributed to chance?
- Under $H_{0}: \lambda = 0.05$ so that $\lambda \times 100 = 5$
- Consider $H_{a}: \lambda > 0.05$

In [71]:
ppois(9, 0.05*100, lower.tail = FALSE)

[1] 0.03182806

3.2% < 5%, it's unlikely for us to have seen as many as 10 infections for a 100 person days at risk

only 3% change of that occuring if in fact, the real infection was 5 for a 100 peson days at risk

so this hospital perhaps should execute those quality control procedures

In [75]:
qt(0.95, 15)

[1] 1.75305

In [3]:
pt(2.5, 15, lower.tail = FALSE)

[1] 0.0122529

In [4]:
pt(2.5, 15, lower.tail = TRUE)

[1] 0.9877471

In [11]:
pbinom(q = 7, size = 8, prob = 0.5, lower.tail = TRUE)

[1] 0.9960938

In [19]:
ppois(9,lambda = 5,lower.tail = FALSE)

[1] 0.03182806

In [18]:
pbinom(9, size = 5, prob = 0.5, lower.tail = FALSE)

[1] 0

In [21]:
1100 + c(-1,1) * qt(0.975, 8) * 10

[1] 1076.94 1123.06

In [35]:
6/qt(0.975, 8, lower.tail = TRUE)

[1] 2.601903

In [33]:
6/pt(0.95, 8)

[1] 7.361625

In [39]:
difference = 3 - 5
t.test(mu = difference, conf.level = 0.95, var.equal = FALSE)
#t.test()

ERROR: Error in t.test.default(mu = difference, conf.level = 0.95, var.equal = FALSE): argument "x" is missing, with no default


In [41]:
-2 + c(-1,1) * qt(0.975, 18) * sqrt((9*0.6 + 9 * 0.68)/18) * (sqrt(2/10))

[1] -2.751649 -1.248351

In [54]:
nx = 100
ny = 100
xbar = 6
ybar = 4
sx = 2
sy = 0.5
alpha = 0.05

sp1 <- sqrt(((nx - 1)*sx^2 + (ny-1)*sy^2)/(nx+ny-2))
sp2 = sqrt(1/nx + 1/ny)
sp = sp1 * sp2

CI <- (xbar - ybar) + c(-1,1) * qt(0.975, 198) * sp
round(CI, 2)

[1] 1.59 2.41

In [55]:
sp

[1] 0.2061553

In [50]:
((nx - 1)*sx^2 + (ny - 1)*sy^2) / (nx + ny - 2)

[1] 1.972017

In [58]:
nx = 9
ny = 9
xbar = -3
ybar = 1
sx = 1.5
sy = 1.8
alpha = 0.1
df = nx + ny -2
sp1 = sqrt(((nx-1)*sx^2 + (ny-1)*sy^2)/df)
sp2 = sqrt(1/nx + 1/ny)
tt = qt(1-alpha/2, df)
CI = (xbar - ybar) + c(-1,1) * tt * sp1 * sp2
round(CI,3)

[1] -5.364 -2.636